In [1]:
from db_connection import run_query

In [2]:
setup_query = """
DROP TABLE IF EXISTS emp;
DROP TABLE IF EXISTS dept;

CREATE TABLE IF NOT EXISTS dept ( 
    deptid INT PRIMARY KEY, 
    deptname VARCHAR(10) NOT NULL
);

CREATE TABLE IF NOT EXISTS emp(
    eid INT PRIMARY KEY,
    ename VARCHAR(50) NOT NULL,
    salary INT CHECK (salary > 0),
    deptid INT NOT NULL,
    FOREIGN KEY(deptid) REFERENCES dept(deptid)
);
"""

for statement in setup_query.strip().split(';'):
    if statement.strip():
        run_query(statement)


Query executed successfully.
Query executed successfully.
Query executed successfully.
Query executed successfully.


In [3]:
insert_query = """
INSERT INTO dept (deptid, deptname) VALUES 
(1 , 'HR'), (2, 'IT'), (3, 'Finance'), (4, 'OPR');

INSERT INTO emp (eid, ename, salary, deptid) VALUES
(1,'a',1000,1), (2,'b',1023,2), (3,'c',23,3), (4,'d',123,3),
(5,'e',102,1), (6,'f',2023,3), (7,'g',323,1), (8,'h',523,2),
(9,'i',136,1), (10,'j',627,1), (11,'k',978,3), (12,'l',222,1),
(13,'m',746,1), (14,'n',3223,1), (15,'o',1999,3), (16,'p',821,3),
(17,'q',324,1), (18,'r',546,3), (19,'s',717,2), (20,'t',817,2),
(21,'u',888,3), (22,'v',9029,1), (23,'w',20001,3), (24,'x',28222,1),
(25,'y',19212,3), (26,'z',12111,1), (27,'a1',3211,1), (28,'a2',9238,1),
(29,'a3',653,3), (30,'a4',44,1), (31,'a5',3,2), (32,'a6',16,1),
(33,'a7',14,2), (34,'a8',845,1), (35,'a9',135,3), (36,'b1',1212,1),
(37,'b2',1242,2), (38,'b3',4212,2), (39,'b4',4444,3), (40,'b5',555,1),
(41,'b6',1553,3), (42,'b7',6432,1), (43,'b8',4532,2), (44,'b9',3355,1),
(45,'c1',2324,2);
"""

for statement in insert_query.strip().split(';'):
    if statement.strip():
        run_query(statement)


Query executed successfully.
Query executed successfully.


`ROW_NUMBER()`

In [4]:
query = """
SELECT e.ename, d.deptname, e.salary,
ROW_NUMBER() OVER(PARTITION BY e.deptid ORDER BY salary DESC) AS row_no
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 10;
"""

run_query(query)


,ename,deptname,salary,row_no
0,x,HR,28222,1
1,z,HR,12111,2
2,a2,HR,9238,3
3,v,HR,9029,4
4,b7,HR,6432,5
5,b9,HR,3355,6
6,n,HR,3223,7
7,a1,HR,3211,8
8,b1,HR,1212,9
9,a,HR,1000,10


`RANK()`

In [5]:
query = """
SELECT e.ename, d.deptname, e.salary,
RANK() OVER(PARTITION BY e.deptid ORDER BY e.salary DESC) AS salrank
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 10;
"""

run_query(query)


,ename,deptname,salary,salrank
0,x,HR,28222,1
1,z,HR,12111,2
2,a2,HR,9238,3
3,v,HR,9029,4
4,b7,HR,6432,5
5,b9,HR,3355,6
6,n,HR,3223,7
7,a1,HR,3211,8
8,b1,HR,1212,9
9,a,HR,1000,10


`RANK()` and `DENSE_RANK()` per dept

In [6]:
query = """
SELECT e.ename, d.deptname, e.salary,
RANK() OVER(PARTITION BY e.deptid ORDER BY e.salary DESC) AS salrank,
DENSE_RANK() OVER(PARTITION BY e.deptid ORDER BY e.salary DESC) AS sal_denserank
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 10;
"""

run_query(query)


,ename,deptname,salary,salrank,sal_denserank
0,x,HR,28222,1,1
1,z,HR,12111,2,2
2,a2,HR,9238,3,3
3,v,HR,9029,4,4
4,b7,HR,6432,5,5
5,b9,HR,3355,6,6
6,n,HR,3223,7,7
7,a1,HR,3211,8,8
8,b1,HR,1212,9,9
9,a,HR,1000,10,10


`RANK()` and `DENSE_RANK()` overall

In [7]:
query = """
SELECT e.ename, d.deptname, e.salary,
RANK() OVER(ORDER BY e.salary) AS salrank1,
DENSE_RANK() OVER(ORDER BY e.salary) AS sal_denserank1
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 10;
"""

run_query(query)


,ename,deptname,salary,salrank1,sal_denserank1
0,a5,IT,3,1,1
1,a7,IT,14,2,2
2,a6,HR,16,3,3
3,c,Finance,23,4,4
4,a4,HR,44,5,5
5,e,HR,102,6,6
6,d,Finance,123,7,7
7,a9,Finance,135,8,8
8,i,HR,136,9,9
9,l,HR,222,10,10


`LEAD()` and `LAG()`

In [8]:
query = """
SELECT ename, deptid, salary,
LEAD(salary, 1) OVER(ORDER BY salary DESC) AS Prev_salary,
LAG(salary, 1) OVER(ORDER BY salary DESC) AS Next_salary
FROM emp
limit 10;
"""
run_query(query)


,ename,deptid,salary,Prev_salary,Next_salary
0,x,1,28222,20001,NaN
1,w,3,20001,19212,28222.0
2,y,3,19212,12111,20001.0
3,z,1,12111,9238,19212.0
4,a2,1,9238,9029,12111.0
5,v,1,9029,6432,9238.0
6,b7,1,6432,4532,9029.0
7,b8,2,4532,4444,6432.0
8,b4,3,4444,4212,4532.0
9,b3,2,4212,3355,4444.0


`Top 3 Salaried Emp per dept`

In [9]:
query = """
SELECT * FROM (
    SELECT e.ename, d.deptname, e.salary,
    DENSE_RANK() OVER(PARTITION BY e.deptid ORDER BY e.salary DESC) AS rank_dense
    FROM emp e JOIN dept d ON e.deptid = d.deptid    
) AS subq
WHERE rank_dense <= 3;
"""
run_query(query)


,ename,deptname,salary,rank_dense
0,x,HR,28222,1
1,z,HR,12111,2
2,a2,HR,9238,3
3,b8,IT,4532,1
4,b3,IT,4212,2
5,c1,IT,2324,3
6,w,Finance,20001,1
7,y,Finance,19212,2
8,b4,Finance,4444,3


`Cumulative SUM()`

In [10]:
query = """
SELECT e.ename, e.deptid, e.salary,
SUM(e.salary) OVER(PARTITION BY e.deptid ORDER BY salary) AS cumsum_sal
FROM emp e
limit 10;
"""
run_query(query)


,ename,deptid,salary,cumsum_sal
0,a6,1,16,16
1,a4,1,44,60
2,e,1,102,162
3,i,1,136,298
4,l,1,222,520
5,g,1,323,843
6,q,1,324,1167
7,b5,1,555,1722
8,j,1,627,2349
9,m,1,746,3095


`Cumulative AVG()`

In [11]:
query = """
SELECT e.ename, e.deptid, e.salary,
AVG(e.salary) OVER(PARTITION BY e.deptid ORDER BY salary) AS cumavg_sal
FROM emp e
limit 10;
"""
run_query(query)


,ename,deptid,salary,cumavg_sal
0,a6,1,16,16.0000
1,a4,1,44,30.0000
2,e,1,102,54.0000
3,i,1,136,74.5000
4,l,1,222,104.0000
5,g,1,323,140.5000
6,q,1,324,166.7143
7,b5,1,555,215.2500
8,j,1,627,261.0000
9,m,1,746,309.5000


`AVG()`

In [40]:
query = """
SELECT e.ename, e.deptid, e.salary,
AVG(e.salary) OVER(PARTITION BY e.deptid) AS avg_sal
FROM emp e;
"""
run_query(query)


,ename,deptid,salary,avg_sal
0,a,1,1000,3855.8571
1,q,1,324,3855.8571
2,a2,1,9238,3855.8571
3,e,1,102,3855.8571
4,g,1,323,3855.8571
5,i,1,136,3855.8571
6,j,1,627,3855.8571
7,l,1,222,3855.8571
8,m,1,746,3855.8571
9,n,1,3223,3855.8571


`COUNT()`

In [42]:
query = """
SELECT d.deptname, COUNT(e.eid) OVER(PARTITION BY e.deptid) AS total_emp_dept
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 1;
"""
run_query(query)


,deptname,total_emp_dept
0,HR,21


`NTILE()`

In [14]:
query = """
SELECT e.ename, d.deptname, e.salary,
NTILE(4) OVER(PARTITION BY e.deptid ORDER BY e.salary) AS salary_bucket
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 10;
"""
run_query(query)


,ename,deptname,salary,salary_bucket
0,a6,HR,16,1
1,a4,HR,44,1
2,e,HR,102,1
3,i,HR,136,1
4,l,HR,222,1
5,g,HR,323,1
6,q,HR,324,2
7,b5,HR,555,2
8,j,HR,627,2
9,m,HR,746,2


`FIRST_VALUE()` AND `LAST_VALUE()`
with default ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW

In [15]:
query = """
SELECT e.ename, d.deptname, e.salary,
FIRST_VALUE(e.salary) OVER(PARTITION BY e.deptid ORDER BY e.salary) AS min_salary,
LAST_VALUE(e.salary) OVER(PARTITION BY e.deptid ORDER BY e.salary) AS max_salary
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 10;
"""
run_query(query)

,ename,deptname,salary,min_salary,max_salary
0,a6,HR,16,16,16
1,a4,HR,44,16,44
2,e,HR,102,16,102
3,i,HR,136,16,136
4,l,HR,222,16,222
5,g,HR,323,16,323
6,q,HR,324,16,324
7,b5,HR,555,16,555
8,j,HR,627,16,627
9,m,HR,746,16,746


`FIRST_VALUE()` AND `LAST_VALUE()`
with ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING

In [16]:
query = """
SELECT e.ename, d.deptname, e.salary,
FIRST_VALUE(e.salary) OVER(PARTITION BY e.deptid ORDER BY e.salary) AS min_salary,
LAST_VALUE(e.salary) OVER(PARTITION BY e.deptid ORDER BY e.salary 
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS max_salary
FROM emp e JOIN dept d ON e.deptid = d.deptid
limit 10;
"""
run_query(query)


,ename,deptname,salary,min_salary,max_salary
0,a6,HR,16,16,28222
1,a4,HR,44,16,28222
2,e,HR,102,16,28222
3,i,HR,136,16,28222
4,l,HR,222,16,28222
5,g,HR,323,16,28222
6,q,HR,324,16,28222
7,b5,HR,555,16,28222
8,j,HR,627,16,28222
9,m,HR,746,16,28222


`PERCENT_RANK()`

In [22]:
query = """
SELECT eid, ename, salary, deptid,
PERCENT_RANK() OVER (PARTITION BY deptid ORDER BY salary) as per_rank 
FROM emp 
LIMIT 10;
"""
run_query(query)

,eid,ename,salary,deptid,per_rank
0,32,a6,16,1,0.00
1,30,a4,44,1,0.05
2,5,e,102,1,0.10
3,9,i,136,1,0.15
4,12,l,222,1,0.20
5,7,g,323,1,0.25
6,17,q,324,1,0.30
7,40,b5,555,1,0.35
8,10,j,627,1,0.40
9,13,m,746,1,0.45


`CUME_DIST()`

In [32]:
query = """SELECT 
    eid,
    ename,
    salary,
    deptid,
    CUME_DIST() OVER (PARTITION BY deptid ORDER BY salary) as cum_dis
FROM emp
limit 10;
"""
run_query(query)

,eid,ename,salary,deptid,cum_dis
0,32,a6,16,1,0.047619
1,30,a4,44,1,0.095238
2,5,e,102,1,0.142857
3,9,i,136,1,0.190476
4,12,l,222,1,0.238095
5,7,g,323,1,0.285714
6,17,q,324,1,0.333333
7,40,b5,555,1,0.380952
8,10,j,627,1,0.428571
9,13,m,746,1,0.476190


In [ ]:
query = """SELECT 
    eid,
    ename,
    salary,
    deptid,
    NTH_VALUE(salary, 3) OVER (PARTITION BY deptid ORDER BY salary DESC 
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS third_highest_salary
FROM emp
limit 10;
"""
run_query(query)